In [1]:
import torch
import torch.nn as nn
import sentencepiece as spm
from encoder_rnn import Encoder
from decoder_rnn import Decoder
from seq2seq_rnn import Seq2Seq


In [2]:
sp_en = spm.SentencePieceProcessor()
sp_en.load("../data/processed/spm_en.model") 

sp_de = spm.SentencePieceProcessor()
sp_de.load("../data/processed/spm_de.model")

True

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [4]:
INPUT_DIM = 16000
OUTPUT_DIM = 16000
ENC_EMB_DIM = 256
DEC_EMB_DIM = 256
HID_DIM = 512

In [ ]:
encoder = Encoder(INPUT_DIM, ENC_EMB_DIM, HID_DIM)
decoder = Decoder(OUTPUT_DIM, DEC_EMB_DIM, HID_DIM)
model = Seq2Seq(encoder, decoder, device).to(device)


In [8]:
model.load_state_dict(torch.load("seq2seq_gru_model.pt", map_location=device))
model.eval()

Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(16000, 256)
    (dropout): Dropout(p=0.5, inplace=False)
    (rnn): GRU(256, 512, batch_first=True)
  )
  (decoder): Decoder(
    (embedding): Embedding(16000, 256)
    (dropout): Dropout(p=0.5, inplace=False)
    (rnn): GRU(256, 512, batch_first=True)
    (fc_out): Linear(in_features=512, out_features=16000, bias=True)
  )
)

In [9]:
def translate_sentence(sentence, model, sp_en, sp_de, device, max_len=50):
    model.eval()
    tokens = sp_en.encode(sentence, out_type=int)
    tokens = [2] + tokens + [3]  # BOS and EOS tokens
    src_tensor = torch.LongTensor(tokens).unsqueeze(0).to(device)  # batch_size=1, seq_len=...

    with torch.no_grad():
        hidden = model.encoder(src_tensor)

    trg_indexes = [2]  # BOS token

    for _ in range(max_len):
        trg_tensor = torch.LongTensor([trg_indexes[-1]]).to(device)
        with torch.no_grad():
            output, hidden = model.decoder(trg_tensor, hidden)
            pred_token = output.argmax(1).item()
        trg_indexes.append(pred_token)
        if pred_token == 3:  # EOS token
            break

    translated_text = sp_de.decode(trg_indexes[1:-1])  # remove BOS and EOS
    return translated_text


In [10]:
english_sentence = "I love you more"
translation = translate_sentence(english_sentence, model, sp_en, sp_de, device)
print("English:", english_sentence)
print("German:", translation)

English: I love you more
German: normalerweise lieben dich mehr.


In [23]:
english_sentence = "I love you more"
translation = translate_sentence(english_sentence, model, sp_en, sp_de, device)
print("English:", english_sentence)
print("German:", translation)

English: I love you more
German: mehr lieben sie lieben lieben!


BLEU score

In [ ]:
# !pip install nltk

You should consider upgrading via the 'D:\AI Projects\LangConv\venv\Scripts\python.exe -m pip install --upgrade pip' command.



  Using cached nltk-3.9.1-py3-none-any.whl (1.5 MB)
  Using cached click-8.2.1-py3-none-any.whl (102 kB)
  Using cached regex-2024.11.6-cp310-cp310-win_amd64.whl (274 kB)
  Using cached tqdm-4.67.1-py3-none-any.whl (78 kB)


In [16]:
import nltk
from nltk.translate.bleu_score import corpus_bleu

nltk.download('punkt')

In [11]:
import pandas as pd

# Path to your file
file_path = "../data/raw/deu-eng/deu.txt"

# Read the file, split by tabs, and keep only the first two columns (English and German)
df = pd.read_csv(file_path, sep="\t", header=None, usecols=[0,1], names=["English", "German"])

df.head()


,English,German
0,Go.,Geh.
1,Hi.,Hallo!
2,Hi.,Grüß Gott!
3,Run!,Lauf!
4,Run.,Lauf!


In [12]:
new_df = df.sample(2500, random_state=42)  # Randomly sample 2000 rows
new_df.reset_index(drop=True, inplace=True)
new_df.head()

,English,German
0,She made her appearance around noon.,Sie hat sich gegen Mittag blicken lassen.
1,I was about to call you.,Ich wollte Sie gerade anrufen.
2,Tom pretended that he was listening.,"Tom tat so, als hörte er zu."
3,Let's cross the street.,Gehen wir über die Straße!
4,It's said that he knows the secret.,"Man sagt, er kenne das Geheimnis."


In [13]:
test_sentences = new_df['English'].tolist()  # list of English sentences (source)
reference_sentences = new_df['German'].tolist()  # list of German sentences (reference)

print(test_sentences[:3])

print(reference_sentences[:3])
# 

['She made her appearance around noon.', 'I was about to call you.', 'Tom pretended that he was listening.']
['Sie hat sich gegen Mittag blicken lassen.', 'Ich wollte Sie gerade anrufen.', 'Tom tat so, als hörte er zu.']


In [78]:
def translate_sentences(sentences, model, sp_en, sp_de, device):
    translations = []
    for sent in sentences:
        # print(f"Translating: {sent}")
        translation = translate_sentence(sent, model, sp_en, sp_de, device)
        # print(f"Translation: {translation}")
        translations.append(translation)
    return translations

In [79]:
input_data = new_df['English'].tolist()  # list of English sentences (source)
input_data[:3]

['She made her appearance around noon.',
 'I was about to call you.',
 'Tom pretended that he was listening.']

In [80]:
#making References
# Original list
refs = new_df['German'].tolist()

# Convert to list of lists (each inner list with one reference string)
references = [[ref] for ref in refs]

print(references[:3])


[['Sie hat sich gegen Mittag blicken lassen.'], ['Ich wollte Sie gerade anrufen.'], ['Tom tat so, als hörte er zu.']]


In [81]:
# candidates are model translations

candidates = translate_sentences(input_data, model, sp_en, sp_de, device)
print("Candidates:", candidates[:3])



Candidates: ['um 1 ⁇  hat man sich um 1 ⁇ .', 'um es war ⁇  uhr.', 'als ha ha ha ha haes haielt als er sich das rennen würden.']


### **BLEU, METEOR, and chrF (with NLTK and SacreBLEU)**

In [23]:
# !pip install sacrebleu

In [82]:
import nltk
from nltk.translate.bleu_score import corpus_bleu
from nltk.translate.meteor_score import meteor_score
from sacrebleu.metrics import CHRF



# Tokenize (basic whitespace tokenization)
tokenized_refs = [[ref.split() for ref in refs] for refs in references]
tokenized_cands = [cand.split() for cand in candidates]



In [85]:
# 1. BLEU Score
from nltk.translate.bleu_score import corpus_bleu, SmoothingFunction


smooth = SmoothingFunction().method1

# Calculate BLEU with weights to only consider 1-gram
bleu_1gram = corpus_bleu(references, candidates, weights=(1, 0, 0, 0) , smoothing_function=smooth)
bleu_2gram = corpus_bleu(references, candidates, weights=(0.5, 0.5, 0, 0), smoothing_function=smooth)
bleu_3gram = corpus_bleu(references, candidates, weights=(0.33, 0.33, 0.33, 0), smoothing_function=smooth)
bleu_4gram = corpus_bleu(references, candidates, weights=(0.25, 0.25, 0.25, 0.25), smoothing_function=smooth)

print(f"1-gram BLEU score: {bleu_1gram:.4f}")
print(f"2-gram BLEU score: {bleu_2gram:.4f}")
print(f"3-gram BLEU score: {bleu_3gram:.4f}")
print(f"4-gram BLEU score: {bleu_4gram:.4f}")

1-gram BLEU score: 0.5732
2-gram BLEU score: 0.3976
3-gram BLEU score: 0.2997
4-gram BLEU score: 0.2322


In [86]:
# 2. CHRF Score

from sacrebleu.metrics import CHRF


chrf_metric = CHRF()
chrf_score = chrf_metric.corpus_score(candidates, references)
print(f"CHRF SCORE: {chrf_score.score:.4f}")



CHRF SCORE: 32.1375
